<a href="https://colab.research.google.com/github/Philliesdog/ML-bootcamp/blob/main/CatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading data

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip


import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-22 18:11:34--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   180MB/s    in 0.4s    

2024-11-22 18:11:34 (180 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [2]:
from PIL import Image
import numpy as np



cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")
i = 0
x_train = []
y_train = []

cat_files = os.listdir(cats_dir)
dog_files = os.listdir(dogs_dir)

while i < len(cat_files):
  im = Image.open(os.path.join(cats_dir, cat_files[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_train.append(np.array(im_resized))
  y_train.append(1)

  im = Image.open(os.path.join(dogs_dir, dog_files[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_train.append(np.array(im_resized))
  y_train.append(0)
  i += 1

x_train = np.array(x_train, dtype="float32")
y_train = np.array(y_train, dtype="int32")

cats_dir_test = os.path.join(validation_dir + "/cats")
dogs_dir_test = os.path.join(validation_dir + "/dogs")
i = 0
x_test = []
y_test = []

cat_files_test = os.listdir(cats_dir_test)
dog_files_test = os.listdir(dogs_dir_test)

while i < len(cat_files_test):
  im = Image.open(os.path.join(cats_dir_test, cat_files_test[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_test.append(np.array(im_resized))
  y_test.append(1)

  im = Image.open(os.path.join(dogs_dir_test, dog_files_test[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_test.append(np.array(im_resized))
  y_test.append(0)
  i += 1

x_test = np.array(x_test, dtype="float32")
y_test = np.array(y_test, dtype="int32")



In [3]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (2000, 150, 150, 3)
y_train shape: (2000,)
x_test shape: (1000, 150, 150, 3)
y_test shape: (1000,)


## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

for layer in pretrained_model.layers:
  layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [5]:
resnet_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,637,826 (93.99 MB)

 Trainable params: 1,050,114 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
history = resnet_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 304ms/step - accuracy: 0.8839 - loss: 0.3722 - val_accuracy: 0.9420 - val_loss: 0.1771
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - accuracy: 0.9676 - loss: 0.0744 - val_accuracy: 0.9470 - val_loss: 0.1882
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.9890 - loss: 0.0318 - val_accuracy: 0.9560 - val_loss: 0.1204
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 95ms/step - accuracy: 0.9943 - loss: 0.0163 - val_accuracy: 0.9550 - val_loss: 0.1646
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 97ms/step - accuracy: 0.9999 - loss: 0.0043 - val_accuracy: 0.9410 - val_loss: 0.2226
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.9972 - loss: 0.0060 - val_accuracy: 0.9480 - val_loss: 0.2052
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.9987 - loss: 0.0055 - val_accuracy: 0.9540 - val_loss: 0.1561
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.9530 

In [31]:
import cv2

def predict_image(image_path):
  image = cv2.imread(image_path)
  image = cv2.resize(image, (150, 150))
  image = np.array(image, dtype="float32")
  image = np.expand_dims(image, axis=0)
  prediction = resnet_model.predict(image)
  print(prediction)
  if prediction[0][0] < prediction[0][1]:
    print("The predicted class is cat")
  else:
    print("The predicted class is dog")

In [32]:

predict_image(os.path.join(cats_dir_test, cat_files_test[0]))
predict_image(os.path.join(dogs_dir_test, dog_files_test[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[1.3254062e-08 1.0000000e+00]]
The predicted class is cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[9.9999964e-01 3.5793548e-07]]
The predicted class is dog
